<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-21 11:03:20
-------------------
qualified stocks: 96
with latest results: 33
still star stocks: 19
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  2.94 L
Current:  1.28 C
-------------------
Today PnL: 92.75 K (0.73%)
Current PnL: -16.35 L (-11.59%)
CY Booked + Current PnL: -12.75 L (-9.04%)
-------------------
Total profit:  4.01 L
Total loss:  -20.36 L
-------------------
Total Booked + Current PnL: 14.23 L (12.48%)
Total Booked PnL: 30.57 L (26.81%)
Curr Year Booked PnL: 3.59 L (2.81%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.07 C
Est FTT PnL: 78.67 L (61.5%)
-------------------
Est LTT:  2.68 C
Est LTT PnL: 1.40 C (109.37%)
Deployed:  1.14 C
Current:  1.28 C
CAGR/XIRR %: 8.22%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ANGELONE,2328.67,3033.00,13.87,H-SC,90.62,102672.0,18840.0,6520.0,2.70,22.47,6.35,30.25,149.0,2.89,0.82,46.72,X40N,NTT,FINANCE
70,SBILIFE,1496.49,1924.99,-0.07,H-LC,69.79,148411.0,22706.0,13283.0,0.51,18.06,8.95,28.63,159.0,1.71,1.19,32.21,AR,ATH,INSURANCE
4,ADANIPORTS,1103.69,1583.00,-1.85,M-LC,81.25,98463.0,20101.0,13933.0,0.16,25.65,14.15,43.43,48.0,1.44,0.79,31.69,XY24,NTT,MISC
63,RELIANCE,1288.53,1526.00,4.98,X-LC,79.17,226256.0,22668.0,14842.0,0.52,11.13,6.56,18.43,161.0,1.53,1.81,23.25,XY25,NTT,REFINERIES
57,PIDILITIND,2504.06,3576.00,-18.89,X-LC,25.00,89856.0,14734.0,17423.0,-0.25,19.61,19.39,42.81,121.0,0.85,0.72,12.69,X40,BTT,CHEMICALS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MASFIN,326.60,399.5,-22.31,H-SC,59.38,88125.0,-9855.0,31725.0,8.14,-10.06,36.00,22.32,102.0,-0.31,0.71,27.81,XR,ATH,FINANCE
84,TITAGARH,1097.23,1548.0,-6.89,H-SC,97.92,163748.0,-28267.0,107157.0,5.29,-14.72,65.44,41.08,119.0,-0.26,1.31,38.59,XY24,NTT,ENGINEERING
21,COFFEEDAY,59.14,80.0,-55.73,L-SC,98.96,64646.0,-48903.0,88953.0,4.99,-43.07,137.60,35.27,181.0,-0.55,0.52,58.22,XR,NTT,HOTELS
92,VIPIND,488.80,718.0,-877.78,H-SC,100.00,74100.0,-21216.0,65912.0,4.30,-22.26,88.95,46.89,173.0,-0.32,0.59,46.15,X40N,NTT,MISC
6,ANGELONE,2328.67,3033.0,13.87,H-SC,90.62,102672.0,18840.0,6520.0,2.70,22.47,6.35,30.25,149.0,2.89,0.82,46.72,X40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,REPCOHOME,515.07,880.00,-56.04,H-SC,21.88,160991.0,-44007.0,189245.0,-3.22,-21.47,117.55,70.85,113.0,-0.23,1.29,29.96,XY24,NTT,FINANCE
44,JCHAC,2282.24,3542.00,19878.26,M-SC,16.67,97614.0,-30191.0,100738.0,-2.37,-23.62,103.20,55.20,210.0,-0.30,0.78,42.49,X40N,BTT,AC
94,WHIRLPOOL,1151.42,2270.00,-35.80,M-SC,93.75,161684.0,15454.0,126599.0,-1.73,10.57,78.30,97.15,28.0,0.12,1.29,38.13,X40,NTT,DURABLES
15,BRITANNIA,4902.95,6446.05,8.87,X-LC,40.62,217120.0,21002.0,40732.0,-1.66,10.71,18.76,31.47,115.0,0.52,1.74,18.64,XY25,ATH,FMCG
41,INDUSINDBK,1354.50,1800.00,-40.59,H-MC,6.25,44061.0,-33146.0,58539.0,-1.18,-42.93,132.86,32.89,44.0,-0.57,0.35,21.34,XR,NTT,BANKS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIAMART,2311.97,4911.36,-51.65,H-SC,55.21,104676.0,2949.0,111428.0,0.65,2.90,106.45,112.43,1.0,0.03,0.84,23.54,AR,ATH,MISC
83,TCS,3928.29,4998.00,-15.52,X-LC,60.42,190717.0,-21411.0,79167.0,0.96,-10.09,41.51,27.23,3.0,-0.27,1.53,9.29,X40,BTT,IT
82,TATAMOTORS,776.02,1065.00,-52.03,M-LC,89.58,192589.0,-13056.0,89631.0,1.09,-6.35,46.54,37.24,4.0,-0.15,1.54,25.36,XY24,NTT,AUTO
10,AWL,342.88,485.00,-69.05,H-MC,3.12,167862.0,-49867.0,140114.0,0.76,-22.90,83.47,41.45,4.0,-0.36,1.34,12.13,XY24,NTT,FMCG
42,INFY,1432.81,2275.00,-12.78,X-LC,67.71,217688.0,18527.0,98547.0,0.35,9.30,45.27,58.78,6.0,0.19,1.74,13.65,X40,BTT,IT


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ANGELONE,2328.67,3033.00,13.87,H-SC,90.62,102672.0,18840.0,6520.0,2.70,22.47,6.35,30.25,149.0,2.89,0.82,46.72,X40N,NTT,FINANCE
58,POLYCAB,5005.95,7541.35,8.17,H-MC,83.33,102774.0,17673.0,25426.0,0.53,20.77,24.74,50.65,56.0,0.70,0.82,29.19,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-1.85,M-LC,81.25,98463.0,20101.0,13933.0,0.16,25.65,14.15,43.43,48.0,1.44,0.79,31.69,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LUXIND,1448.19,2442.00,43.68,M-SC,66.67,149017.0,11439.0,82973.0,0.20,8.31,55.68,68.62,66.0,0.14,1.19,29.19,X40N,NTT,TEXTILES
6,ANGELONE,2328.67,3033.00,13.87,H-SC,90.62,102672.0,18840.0,6520.0,2.70,22.47,6.35,30.25,149.0,2.89,0.82,46.72,X40N,NTT,FINANCE
20,COALINDIA,406.65,512.15,8.40,L-LC,43.75,151404.0,944.0,38093.0,0.27,0.63,25.16,25.94,85.0,0.02,1.21,15.53,XY25,ATH,MINING
58,POLYCAB,5005.95,7541.35,8.17,H-MC,83.33,102774.0,17673.0,25426.0,0.53,20.77,24.74,50.65,56.0,0.70,0.82,29.19,X40N,ATH,CABLES
29,GILLETTE,8109.44,10330.69,1.61,H-SC,84.38,149889.0,12029.0,25736.0,1.30,8.73,17.17,27.39,64.0,0.47,1.20,31.21,X40,ATH,FMCG


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,-2.67,H-SC,17.71,109410.0,7406.0,37615.0,1.05,7.26,34.38,44.14,31.0,0.20,0.88,24.68,XR,ATH,MISC
12,BANKINDIA,113.49,190.00,-31.87,M-MC,19.79,133681.0,4529.0,82535.0,1.12,3.51,61.74,67.42,57.0,0.05,1.07,30.02,XR,NTT,BANKS
95,WIPRO,243.46,420.00,-8.80,M-LC,56.25,155930.0,4985.0,104473.0,0.72,3.30,67.00,72.51,49.0,0.05,1.25,17.40,XR,NTT,IT
39,INDIAMART,2311.97,4911.36,-51.65,H-SC,55.21,104676.0,2949.0,111428.0,0.65,2.90,106.45,112.43,1.0,0.03,0.84,23.54,AR,ATH,MISC
34,HEROMOTOCO,4311.81,6039.03,-1.90,H-MC,74.48,149401.0,-1512.0,61972.0,0.64,-1.00,41.48,40.06,27.0,-0.02,1.20,22.20,AR,ATH,AUTO


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,59.14,80.0,-55.73,L-SC,98.96,64646.0,-48903.0,88953.0,4.99,-43.07,137.60,35.27,181.0,-0.55,0.52,58.22,XR,NTT,HOTELS
53,MASFIN,326.60,399.5,-22.31,H-SC,59.38,88125.0,-9855.0,31725.0,8.14,-10.06,36.00,22.32,102.0,-0.31,0.71,27.81,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,-1.85,M-LC,81.25,98463.0,20101.0,13933.0,0.16,25.65,14.15,43.43,48.0,1.44,0.79,31.69,XY24,NTT,MISC
6,ANGELONE,2328.67,3033.0,13.87,H-SC,90.62,102672.0,18840.0,6520.0,2.70,22.47,6.35,30.25,149.0,2.89,0.82,46.72,X40N,NTT,FINANCE
90,VALIANTORG,512.64,838.0,-344.54,H-SC,71.35,103831.0,-29455.0,114048.0,0.53,-22.10,109.84,63.47,145.0,-0.26,0.83,74.05,XR,NTT,CHEMICALS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,718.00,-877.78,H-SC,100.00,74100.0,-21216.0,65912.0,4.30,-22.26,88.95,46.89,173.0,-0.32,0.59,46.15,X40N,NTT,MISC
21,COFFEEDAY,59.14,80.00,-55.73,L-SC,98.96,64646.0,-48903.0,88953.0,4.99,-43.07,137.60,35.27,181.0,-0.55,0.52,58.22,XR,NTT,HOTELS
84,TITAGARH,1097.23,1548.00,-6.89,H-SC,97.92,163748.0,-28267.0,107157.0,5.29,-14.72,65.44,41.08,119.0,-0.26,1.31,38.59,XY24,NTT,ENGINEERING
51,LTIM,5664.39,7262.39,-5.69,X-LC,96.88,151500.0,-18432.0,66372.0,0.69,-10.85,43.81,28.21,32.0,-0.28,1.21,24.47,X200,ATH,IT
80,TANLA,917.30,1963.11,-39.41,H-SC,95.83,148144.0,-91271.0,364227.0,0.94,-38.12,245.86,114.01,20.0,-0.25,1.19,35.87,AR,ATH,IT


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,17.28
1,25,38.55
2,50,67.72


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.31
X40N    14.39
XY25    14.05
X40     13.48
AR      11.38
XR      10.05
X200     1.21
SR       1.12
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.52
M-SC    17.94
X-LC    16.22
H-MC     8.45
M-LC     7.69
H-LC     6.03
M-MC     5.63
X-MC     5.61
X-SC     2.87
L-SC     2.35
L-LC     1.21
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.71,-1.25,33.47
BANKS,11.07,-8.86,62.25
IT,9.23,-20.01,86.97
FINANCE,6.93,-19.95,75.25
MISC,6.52,-30.34,86.39
INSURANCE,6.22,0.75,29.17
PAINTS,5.58,-18.09,53.04
AUTO,4.39,-10.32,58.41
ELECTRICAL,4.18,-6.45,60.69


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2957238.0,27
X40N,1195105.0,18
AR,1170894.0,13
XR,1093749.0,14
X40,719961.0,11
XY25,459125.0,10
SR,205017.0,2
X200,66372.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2289011.0,22
M-SC,1998711.0,21
X-LC,714451.0,12
M-MC,562950.0,5
H-MC,468296.0,8
X-SC,348639.0,4
M-LC,314369.0,6
L-SC,303726.0,4
X-MC,302098.0,5


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      898902.0      6
M-SC       XY24      752374.0      6
H-SC       AR        535673.0      3
           X40N      510692.0      7
M-SC       X40N      447368.0      6
M-MC       XY24      397416.0      3
X-LC       X40       384378.0      5
M-SC       XR        301879.0      4
H-SC       XR        274165.0      4
M-SC       X40       197394.0      2
M-LC       XY24      190208.0      4
H-LC       AR        189467.0      4
H-MC       XY24      181104.0      2
L-SC       XR        154113.0      2
X-LC       AR        143890.0      2
X-MC       XY24      142958.0      2
M-SC       XY25      122222.0      1
X-SC       SR        120321.0      1
X-LC       XY25      119811.0      4
X-SC       XR        118045.0      1
           X40N      110273.0      2
M-LC       XR        104473.0      1
X-MC       X40N      101346.0      2
M-SC       AR         92778.0      1
L-SC       XY24       85498.0      1
M-SC       SR         84696.0      1
M-MC       AR         82999.0      1
           XR         82535.0      1
H-MC       XY25       72645.0      1
H-SC       X40        69579.0      2
H-MC       X40        68610.0      2
X-LC       X200       66372.0      1
L-SC       AR         64115.0      1
H-MC       AR         61972.0      1
           XR         58539.0      1
X-MC       XY25       57794.0      1
L-LC       XY25       38093.0      1
H-LC       XY25       28872.0      1
H-MC       X40N       25426.0      1
M-LC       XY25       19688.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 35.0 seconds
